<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# Modification of the span of an aircraft.

This notebook is based on the 3rd FAST-OAD-GA tutorial written by Aurélien. It aims to compare a reference architecture represented by the .XML file of an aircraft (Beechcraft 76, Cirrus 22, TBM-900) with a modified configuration. The modified architecture is produced by the FAST-OAD-GA module called modify_config which creates the corresponding .XML file. Then the .xml files of the reference aircraft and of the modified one go through all of the FAST-OAD-GA modules, with the generate_block_analysis function which calls each module. The variable viewer is called for both aircrafts after each module.

The only modification implemented for now is the increase of the wing span. The user has to define the multiplier of the span, and can also play on other parameters such as setting the engine position along the span.

The fastga code is from the root analysis_mode on GitHub.

Like in the tutorials, the structure of the notebook is based on data transfers between the data folder which stocks the .xml of the reference aircraft, and the workdir folder which stocks the .xml files passing through the modules of FAST-OAD-GA. After all the modules have been executed the final .xml files are placed in the output folder.

As the TBM-900 turbopropeller is still being modelized in FAST-OAD-GA, the Beechcraft 76 will be used for this analysis. There is no limit in the code for the increase in span, but some modules could have problems running with an extreme value (such as two times the original span).

## 1. Initialization of the work environment

In [1]:
import os.path as pth
import os
import openmdao.api as om
from fastoad import api as api_cs25
from fastga.command import api as api_cs23
import logging
from fastoad.gui import VariableViewer
import shutil

# Define relative path.
current_path = os.getcwd()
DATA_FOLDER_PATH = pth.join(current_path, 'data')
WORK_FOLDER_PATH = pth.join(current_path, 'workdir')
OUTPUT_FOLDER_PATH = pth.join(current_path, 'output')

# Clear work folder.
shutil.rmtree(WORK_FOLDER_PATH, ignore_errors=True)
os.mkdir(WORK_FOLDER_PATH)

# For using more screen width.
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# Final file names. These .xml files will be overwritten each time they pass through a module.
AIRCRAFT_REF_FILE = pth.join(WORK_FOLDER_PATH, 'geometry_reference.xml')
AIRCRAFT_MOD_FILE = pth.join(WORK_FOLDER_PATH, 'geometry_mod.xml')

# Copy the reference aircraft file in the files of the work folder.
shutil.copy(pth.join(DATA_FOLDER_PATH, 'TBM-900.xml'), AIRCRAFT_REF_FILE)
shutil.copy(pth.join(DATA_FOLDER_PATH, 'beechcraft_76.xml'), AIRCRAFT_MOD_FILE)

Failed to import module fastga.models.weight.cg.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.loops.unitary_tests.test_wing_area.py
Failed to import module fastga.models.aerodynamics.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.propulsion.fuel_propulsion.basicIC_engine.unitary_tests.test_openmdao_engine.py
Failed to import module fastga.models.handling_qualities.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.performances.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.load_analysis.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.geometry.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.aerodynamics.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.load_analysis.fuselage.compute_bending_moment_reversed_old.py
Failed to import module fastga.models.geometry.unitary_tests.test_beechcraft_76.py
Failed to import module fa

'C:\\Users\\Lucas\\FAST-OAD-GA\\FAST-GA\\src\\fastga\\notebooks\\Lucas\\workdir\\geometry_mod.xml'

## Propeller Computation

In [2]:
from fastga.models.aerodynamics.components.compute_propeller_aero import ComputePropellerPerformance

compute_propeller_ref = api_cs23.generate_block_analysis(
        ComputePropellerPerformance(),
        [],
        str(AIRCRAFT_REF_FILE),
        True,
    )

# compute_propeller_mod = api_cs23.generate_block_analysis(
#         ComputePropellerPerformance(),
#         [],
#         str(AIRCRAFT_MOD_FILE),
#         True,
#     )


output_ref = compute_propeller_ref({})
# output_mod = compute_propeller_mod({})

# Open viewer
api_cs25.variable_viewer(AIRCRAFT_REF_FILE)

Entering propeller computation


2 Explicit Output(s) in 'system.propeller_aero_conditions'

varname                               value     
------------------------------------  ----------
data:aerodynamics:propeller:mach      [0.]      
data:aerodynamics:propeller:reynolds  [1000000.]


0 Implicit Output(s) in 'system.propeller_aero_conditions'




C:\Users\Lucas\FAST-OAD-GA\FAST-GA\src\fastga\models\aerodynamics\components\compute_propeller_aero.py:160: RuntimeWarning: divide by zero encountered in true_divide
  (v_inf + 60.0 / v_inf) / (omega * 2.0 * math.pi / 60.0 * radius_ratio_vect)
C:\Users\Lucas\FAST-OAD-GA\FAST-GA\src\fastga\models\aerodynamics\components\compute_propeller_aero.py:358: UserWarning: Propeller not working in propulsive mode!
  warnings.warn("Propeller not working in propulsive mode!")
c:\users\lucas\fast-oad-ga\venvtest\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)
C:\Users\Lucas\FAST-OAD-GA\FAST-GA\src\fastga\models\aerodynamics\components\compute_propeller_aero.py:346: UserWarning: Propeller element out of calculated polars: contribution canceled!
  warnings.warn("Propeller element out of calculated polars: contribution canceled!")
C:

## 2. Definition of the modifications from the reference architecture

Describes the different parameters the user has to choose in order to obtain the desired modified configuration. The function generate_block_analysis is then used twice to create the .xml file related to the modified configuration.

In [ ]:
# Span modifications
span_length_multiplier = 1.1       # value by which the wing span is multiplied. If 1.0, the span doesn't change.
fixed_engine_position = True       # boolean stating if the engines stay at the same y-position along the span or if their y-ratio is conserved
fuel_added_part = False            # boolean stating if fuel tanks are added in the extended part of the wing. If True, only applies for y_ratio_tank_end =1.0 in the initial .xml file.


In [ ]:
from fastga.models.modify_config import ComputeConfigMod


compute_modify = api_cs23.generate_block_analysis(
        ComputeConfigMod(span_mod=[span_length_multiplier, fixed_engine_position, fuel_added_part]),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )

output_mod = compute_modify({})

# Open viewer
api_cs25.variable_viewer(AIRCRAFT_MOD_FILE)

In [ ]:
from fastga.models.modify_config.update_XML import UpdateXML


compute_update = api_cs23.generate_block_analysis(
        UpdateXML(span_mod=[span_length_multiplier, fixed_engine_position, fuel_added_part]),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )

output_mod = compute_update({})

# Open viewer
api_cs25.variable_viewer(AIRCRAFT_MOD_FILE)

## 3. Geometry Module

In [3]:
from fastga.models.geometry import GeometryFixedTailDistance as Geometry

# Declare function
compute_geometry_ref = api_cs23.generate_block_analysis(
        Geometry(propulsion_id="fastga.wrapper.propulsion.basic_turboprop"),
        [],
        str(AIRCRAFT_REF_FILE),
        True,
    )

# compute_geometry_mod = api_cs23.generate_block_analysis(
#         Geometry(
#             propulsion_id="fastga.wrapper.propulsion.basicIC_engine"),
#         [],
#         str(AIRCRAFT_MOD_FILE),
#         True,
#     )

# Compute geometry
output_ref = compute_geometry_ref({})
# output_mod = compute_geometry_mod({})

Variable "data:propulsion:IC_engine:max_power" out of bound: value [625600.] W is over upper limit ( 250000 ) in file C:\Users\Lucas\FAST-OAD-GA\FAST-GA\src\fastga\models\propulsion\fuel_propulsion\basicIC_engine\openmdao.py
Variable "data:propulsion:IC_engine:max_power" out of bound: value [625600.] W is over upper limit ( 250000 ) in file C:\Users\Lucas\FAST-OAD-GA\FAST-GA\src\fastga\models\propulsion\fuel_propulsion\turboprop_simple\openmdao.py


In [ ]:
# Open viewer
api_cs25.variable_viewer(AIRCRAFT_REF_FILE)

In [ ]:
api_cs25.variable_viewer(AIRCRAFT_MOD_FILE)

## 4. Aerodynamics Module 

We choose to use the software OpenVSP, and we activate the mach interpolation option because it is needed to execute the load analysis module. We also compute the slipstream which allows to get more variables in cruise and to plot the lift coefficient along the span in the post-processing notebook.

This module usually represents the longest computational time.

In [4]:
from fastga.models.aerodynamics.aerodynamics import Aerodynamics

# Declare function on reference file
compute_aero_ref = api_cs23.generate_block_analysis(
        Aerodynamics(
            propulsion_id="fastga.wrapper.propulsion.basic_turboprop",
            use_openvsp=True,
            compute_mach_interpolation=True,
            compute_slipstream_cruise=True,
#             wing_airfoil_file="naca43013_3.af"
        ),
        [],
        str(AIRCRAFT_REF_FILE),
        True,
    )


# # Declare function on modified file
# compute_aero_mod = api_cs23.generate_block_analysis(
#         Aerodynamics(
#             propulsion_id="fastga.wrapper.propulsion.basicIC_engine",
#             use_openvsp=True,
#             compute_mach_interpolation=True,
#             compute_slipstream_cruise=True,
# #             wing_airfoil_file="naca43013_3.af"
#         ),
#         [],
#         str(AIRCRAFT_MOD_FILE),
#         True,
#     )

# Compute both aircraft designs
output_ref = compute_aero_ref({})
# output_mod = compute_aero_mod({})

# # Save Results
# OUTPUT_AERO_FILE = pth.join(OUTPUT_FOLDER_PATH, 'output_ref_aero.xml')
# OUTPUT_MOD_FILE = pth.join(OUTPUT_FOLDER_PATH, 'output_mod_aero.xml')
# shutil.copy(AIRCRAFT_REF_FILE, OUTPUT_AERO_FILE)
# shutil.copy(AIRCRAFT_MOD_FILE, OUTPUT_MOD_FILE)

C:\Users\Lucas\FAST-OAD-GA\FAST-GA\src\fastga\models\aerodynamics\external\xfoil\xfoil_polar.py:421: UserWarning: genfromtxt: Empty input file: "C:\Users\Lucas\AppData\Local\Temp\x3rlo9xbh\out"
  result_array = np.genfromtxt(xfoil_result_file_path, skip_header=12, dtype=dtypes)
2D CL max not found, less than 40% of angle range computed: using default value 1.9
Variable "data:propulsion:IC_engine:max_power" out of bound: value [625600.] W is over upper limit ( 250000 ) in file C:\Users\Lucas\FAST-OAD-GA\FAST-GA\src\fastga\models\propulsion\fuel_propulsion\basicIC_engine\openmdao.py
Variable "data:propulsion:IC_engine:max_power" out of bound: value [625600.] W is over upper limit ( 250000 ) in file C:\Users\Lucas\FAST-OAD-GA\FAST-GA\src\fastga\models\propulsion\fuel_propulsion\turboprop_simple\openmdao.py


In [ ]:
api_cs25.variable_viewer(AIRCRAFT_REF_FILE)

In [ ]:
api_cs25.variable_viewer(AIRCRAFT_MOD_FILE)

## 5. Weight Module (+ update MTOW)

In [5]:
from fastga.models.weight import Weight

# Declare function on reference file
compute_weight_ref = api_cs23.generate_block_analysis(
        Weight(
            propulsion_id="fastga.wrapper.propulsion.basic_turboprop",
        ),
        [],
        str(AIRCRAFT_REF_FILE),
        True,
    )


# # Declare function on modified file
# compute_weight_mod = api_cs23.generate_block_analysis(
#         Weight(
#             propulsion_id="fastga.wrapper.propulsion.basicIC_engine",
#         ),
#         [],
#         str(AIRCRAFT_MOD_FILE),
#         True,
#     )

output_ref = compute_weight_ref({})
# output_mod = compute_weight_mod({})

Variable "data:propulsion:IC_engine:max_power" out of bound: value [625600.] W is over upper limit ( 250000 ) in file C:\Users\Lucas\FAST-OAD-GA\FAST-GA\src\fastga\models\propulsion\fuel_propulsion\basicIC_engine\openmdao.py
Variable "data:propulsion:IC_engine:max_power" out of bound: value [625600.] W is over upper limit ( 250000 ) in file C:\Users\Lucas\FAST-OAD-GA\FAST-GA\src\fastga\models\propulsion\fuel_propulsion\turboprop_simple\openmdao.py


In [ ]:
api_cs25.variable_viewer(AIRCRAFT_REF_FILE)

In [ ]:
api_cs25.variable_viewer(AIRCRAFT_MOD_FILE)

In [6]:
from fastga.models.weight.mass_breakdown.update_mtow import UpdateMTOW

var_inputs = ["data:mission:sizing:fuel"]

compute_mtow_ref = api_cs23.generate_block_analysis(
        UpdateMTOW(),
        var_inputs,
        str(AIRCRAFT_REF_FILE),
        True,
    )

# compute_mtow_mod = api_cs23.generate_block_analysis(
#         UpdateMTOW(),
#         var_inputs,
#         str(AIRCRAFT_MOD_FILE),
#         True,
#     )

inputs_dict = {"data:mission:sizing:fuel": (150., "kg")}

output_ref = compute_mtow_ref(inputs_dict)
# output_mod = compute_mtow_mod(inputs_dict)

Variable "data:propulsion:IC_engine:max_power" out of bound: value [625600.] W is over upper limit ( 250000 ) in file C:\Users\Lucas\FAST-OAD-GA\FAST-GA\src\fastga\models\propulsion\fuel_propulsion\basicIC_engine\openmdao.py
Variable "data:propulsion:IC_engine:max_power" out of bound: value [625600.] W is over upper limit ( 250000 ) in file C:\Users\Lucas\FAST-OAD-GA\FAST-GA\src\fastga\models\propulsion\fuel_propulsion\turboprop_simple\openmdao.py


In [ ]:
api_cs25.variable_viewer(AIRCRAFT_REF_FILE)

In [ ]:
api_cs25.variable_viewer(AIRCRAFT_MOD_FILE)

## 6. Performances Module

In [7]:
from fastga.models.performances import Mission

compute_sizing_ref = api_cs23.generate_block_analysis(
        Mission(
            propulsion_id="fastga.wrapper.propulsion.basic_turboprop",
        ),
        [],
        str(AIRCRAFT_REF_FILE),
        True,
    )

# compute_sizing_mod = api_cs23.generate_block_analysis(
#         Mission(
#             propulsion_id="fastga.wrapper.propulsion.basicIC_engine",
#         ),
#         [],
#         str(AIRCRAFT_MOD_FILE),
#         True,
#     )

output_ref = compute_sizing_ref({})
# output_mod = compute_sizing_mod({})

# # Save Results
# OUTPUT_REF_FILE = pth.join(OUTPUT_FOLDER_PATH, 'output_ref_perf.xml')
# OUTPUT_MOD_FILE = pth.join(OUTPUT_FOLDER_PATH, 'output_mod_perf.xml')
# shutil.copy(AIRCRAFT_REF_FILE, OUTPUT_REF_FILE)
# shutil.copy(AIRCRAFT_MOD_FILE, OUTPUT_MOD_FILE)

Variable "data:propulsion:IC_engine:max_power" out of bound: value [625600.] W is over upper limit ( 250000 ) in file C:\Users\Lucas\FAST-OAD-GA\FAST-GA\src\fastga\models\propulsion\fuel_propulsion\basicIC_engine\openmdao.py
Variable "data:propulsion:IC_engine:max_power" out of bound: value [625600.] W is over upper limit ( 250000 ) in file C:\Users\Lucas\FAST-OAD-GA\FAST-GA\src\fastga\models\propulsion\fuel_propulsion\turboprop_simple\openmdao.py


In [ ]:
api_cs25.variable_viewer(AIRCRAFT_REF_FILE)

In [ ]:
api_cs25.variable_viewer(AIRCRAFT_MOD_FILE)

## 7. Load Analysis Module

In [8]:
from fastga.models.load_analysis.loads import Loads

compute_loads_ref = api_cs23.generate_block_analysis(
        Loads(
            propulsion_id="fastga.wrapper.propulsion.basic_turboprop",
        ),
        [],
        str(AIRCRAFT_REF_FILE),
        True,
    )

# compute_loads_mod = api_cs23.generate_block_analysis(
#         Loads(
#             propulsion_id="fastga.wrapper.propulsion.basicIC_engine",
#         ),
#         [],
#         str(AIRCRAFT_MOD_FILE),
#         True,
#     )

output_ref = compute_loads_ref({})
# output_mod = compute_loads_mod({})

C:\Users\Lucas\FAST-OAD-GA\FAST-GA\src\fastga\models\load_analysis\fuselage\compute_bending_moment.py:163: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  load_case_array = np.array([[n_lift, lift_max_h], [n_emergency_landing, 0]])
Variable "data:propulsion:IC_engine:max_power" out of bound: value [625600.] W is over upper limit ( 250000 ) in file C:\Users\Lucas\FAST-OAD-GA\FAST-GA\src\fastga\models\propulsion\fuel_propulsion\basicIC_engine\openmdao.py
Variable "data:propulsion:IC_engine:max_power" out of bound: value [625600.] W is over upper limit ( 250000 ) in file C:\Users\Lucas\FAST-OAD-GA\FAST-GA\src\fastga\models\propulsion\fuel_propulsion\turboprop_simple\openmdao.py


In [ ]:
api_cs25.variable_viewer(AIRCRAFT_REF_FILE)

In [ ]:
api_cs25.variable_viewer(AIRCRAFT_MOD_FILE)

## 8. Handling Qualities Module

As the main purpose of this notebook is to directly visualize the variations of the plane performance and its handling qualities by modifying the reference architecture, the static margin will be computed without updating the tail areas.

In [9]:
from fastga.models.handling_qualities.handling_qualities import ComputeHandlingQualities

compute_hq_ref = api_cs23.generate_block_analysis(
        ComputeHandlingQualities(
            propulsion_id="fastga.wrapper.propulsion.basic_turboprop",
        ),
        [],
        str(AIRCRAFT_REF_FILE),
        True,
    )

# compute_hq_mod = api_cs23.generate_block_analysis(
#         ComputeHandlingQualities(
#             propulsion_id="fastga.wrapper.propulsion.basicIC_engine",
#         ),
#         [],
#         str(AIRCRAFT_MOD_FILE),
#         True,
#     )

output_ref = compute_hq_ref({})
# output_mod = compute_hq_mod({})

Variable "data:propulsion:IC_engine:max_power" out of bound: value [625600.] W is over upper limit ( 250000 ) in file C:\Users\Lucas\FAST-OAD-GA\FAST-GA\src\fastga\models\propulsion\fuel_propulsion\basicIC_engine\openmdao.py
Variable "data:propulsion:IC_engine:max_power" out of bound: value [625600.] W is over upper limit ( 250000 ) in file C:\Users\Lucas\FAST-OAD-GA\FAST-GA\src\fastga\models\propulsion\fuel_propulsion\turboprop_simple\openmdao.py


In [ ]:
api_cs25.variable_viewer(AIRCRAFT_REF_FILE)

In [ ]:
api_cs25.variable_viewer(AIRCRAFT_MOD_FILE)

## Save files for post processing

In [ ]:
# Clear output folder.
shutil.rmtree(OUTPUT_FOLDER_PATH, ignore_errors=True)
os.mkdir(OUTPUT_FOLDER_PATH)

OUTPUT_REF_FILE = pth.join(OUTPUT_FOLDER_PATH, 'output_ref.xml')
OUTPUT_MOD_FILE = pth.join(OUTPUT_FOLDER_PATH, 'output_mod.xml')
shutil.copy(AIRCRAFT_REF_FILE, OUTPUT_REF_FILE)
shutil.copy(AIRCRAFT_MOD_FILE, OUTPUT_MOD_FILE)